In [ ]:
from shapely import wkt, affinity
import shapely
import pandas as pd
import json, geojson

cls = ['Buildings','Misc.', 'Road' , 'Track', 'Trees', 'Crops', 'Waterway', 'Standing water', 'Vehicle Large', 'Vehicle Small']
cls_col = ['gold','purple', 'gray' , 'brown', 'green', 'yellow', 'blue', 'darkgreen', 'orange', 'red']

g = pd.read_csv('../input/grid_sizes.csv', names=['ImageId','Xmax','Ymin'])
w = pd.read_csv('../input/train_wkt_v4.csv')
d = {}
for i in range(len(g)):
    d[str(g['ImageId'][i])+'.tif'] = {'x': g['Xmax'][i], 'y': g['Ymin'][i], 'wkt':['','','','','','','','','','']}
for i in range(len(w)):
    d[str(w['ImageId'][i])+'.tif']['wkt'][w['ClassType'][i] -1 ] = w['MultipolygonWKT'][i]

X_ = 400 * (400/(400+1))

html = open('output.html', 'w')
html.write("""
<!DOCTYPE html>
<html>
<body>
""")

im_break = 0
for img in d:
    if d[img]['wkt'] != ['','','','','','','','','','']:
        html.write("<br/><br/>"+ img + "<br/><svg height='400' width='400'>")
        i = 0
        for poly_ in d[img]['wkt']:
            if len(poly_)>0:
                poly = wkt.loads(poly_)
                poly = affinity.scale(poly, xfact= X_/float(d[img]['x']), yfact= X_/float(d[img]['y']), origin=(0,0,0))
                for p in poly:
                    html.write("<polygon points='" + " ".join([str(x[0])+','+str(x[1]) for x in p.exterior.coords]) + "' style='fill:" + cls_col[i] + ";stroke:black;stroke-width:1;' />")
            i+=1
        html.write("</svg>")
        im_break += 1
    if im_break > 2: #remove
        break #remove
html.write("</body></html>")
html.close()

In [ ]:
from IPython.core.display import HTML
HTML(open('output.html', 'r').read())